In [1]:
from model_functions import *
from param_functions import *
import numpy as np
import pandas as pd
from IPython import display as ipd
from matplotlib import pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 100

%matplotlib qt


Bad key "text.kerning_factor" on line 4 in
C:\Users\Adrien\anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


--------------------

## Original Setup - Drive & Flux

### Image

In [ ]:
ipd.Image('D:\\Uni\\Honours\\GitLab\\DiLonardo_Adrien\\' +
            '2021 Honours - Dilution Fridge\\Numerical Modeling\\' +
            'Optimisation Figures and Code\\Cable Setup Images\\Original.PNG')

### Data

In [ ]:
cables = ['219-SS-SS', '219-CuNi-CuNi', '219-AgSS-SS', '219-AgCuNi-CuNi',
          '119-SS-SS', '119-CuNi-CuNi', '119-AgSS-SS', '119-AgCuNi-CuNi']

therm_scheme_drive = [[False, True, False, True, True], #inner
                  [False, True, False, True, True], #dielectric
                  [True, True, True, True, True]]   #outer

therm_scheme_flux = [[False, True, False, False, True], #inner
                  [False, True, False, False, True], #dielectric
                  [True, True, True, True, True]]   #outer

In [ ]:
data = []

for c in cables:
    p_drive = passive_load(stage_labels=fridge_ours['labels'], 
                diameters=coax_co_params[c]['d'],
                lengths=fridge_ours['lengths'],
                therm_cond=[coax_co_params[c]['tc'], lambda t: 0, coax_co_params[c]['tc']],
                therm_scheme=therm_scheme_drive,
                stage_temps=fridge_ours['temps'])

    p_flux = passive_load(stage_labels=fridge_ours['labels'], 
                diameters=coax_co_params[c]['d'],
                lengths=fridge_ours['lengths'],
                therm_cond=[coax_co_params[c]['tc'], lambda t: 0, coax_co_params[c]['tc']],
                therm_scheme=therm_scheme_flux,
                stage_temps=fridge_ours['temps'])

    a_ac = active_load_AC(stage_labels=fridge_ours['labels'], 
            signal_p=1e-3*10**(-5/10), 
            signal_f=6, 
            att=[0, 20, 0, 20, 20],
            cable_att=coax_co_params[c]['att_4'],
            lengths=fridge_ours['lengths'])

    a_dc = active_load_DC(i_in=20e-3, 
                    stage_labels=fridge_ours['labels'], 
                    att=[0, 20, 0, 0, 0], 
                    cable_rho=coax_co_params[c]['rho'], 
                    lengths=fridge_ours['lengths'], 
                    diameters=coax_co_params[c]['d'])

    data.append([p_drive, p_flux, a_ac, a_dc])

### Plot

In [ ]:
n_drive = 14
n_flux = 14
dc_drive = 0.33
dc_flux = 1

edited_loads = [[di[0]*n_drive, 
                 di[1]*n_flux, 
                 di[2]*dc_drive*n_drive, 
                 di[3]*dc_flux*n_flux]
                for di in data]

figs = []
colours = ['red', 'orange', 'green', 'blue']

#iterating stages
for i in range(5):

    fig = plt.figure(figsize=(8, 6))
    figs.append(fig)
    ax = plt.axes()

    #iterating cables
    for ii in range(4):
        #iterating load type
        bot = 0
        for iii in range(4):
            val = edited_loads[ii][iii][i] / fridge_ours['cool_p'][i]
            plt.bar(x=1+ii, 
                    height=val, 
                    width=1, 
                    bottom=bot,
                    color=colours[iii])
            bot += val

    #iterating cables
    for ii in range(4, 8):
        #iterating load type
        bot = 0
        for iii in range(4):
            val = edited_loads[ii][iii][i] / fridge_ours['cool_p'][i]
            plt.bar(x=2+ii, 
                    height=val, 
                    width=1, 
                    bottom=bot,
                    color=colours[iii])
            bot += val

    plt.title('Heat Loads on ' + fridge_100q['labels'][i] + ' Stage\n' +
                'No. Drive = ' + str(n_drive) + ', No. Flux = ' + str(n_flux))
    plt.xlabel('Cable Types')
    plt.ylabel(r'$P_{total}/P_{cooling}$')
    leg_labels = ['Passive (Drive)', 'Passive (Flux)', 'Active (Drive)', 'Active (Flux)']
    leg = plt.legend(leg_labels)
    plt.legend(leg.legendHandles[::-1], leg_labels[::-1], loc=(1.01, 0.75))
    plt.xticks([1, 2, 3, 4, 6, 7, 8, 9], cables, rotation=90)

plt.show()

## 10 dB from 4K to 50K - Drive & Flux

### Image

In [ ]:
ipd.Image('D:\\Uni\\Honours\\GitLab\\DiLonardo_Adrien\\' +
            '2021 Honours - Dilution Fridge\\Numerical Modeling\\' +
            'Optimisation Figures and Code\\Cable Setup Images\\10db 50K.PNG')

### Data

In [ ]:
cables = ['219-SS-SS', '219-CuNi-CuNi', '219-AgSS-SS', '219-AgCuNi-CuNi',
          '119-SS-SS', '119-CuNi-CuNi', '119-AgSS-SS', '119-AgCuNi-CuNi']

therm_scheme_drive_10 = [[True, True, False, True, True], #inner
                  [True, True, False, True, True], #dielectric
                  [True, True, True, True, True]]   #outer

therm_scheme_flux_10 = [[True, True, False, False, True], #inner
                  [True, True, False, False, True], #dielectric
                  [True, True, True, True, True]]   #outer

In [ ]:
heat_data_10 = []

for c in cables:
    p_drive = passive_load(stage_labels=fridge_100q['labels'], 
                diameters=coax_co_params[c]['d'],
                lengths=fridge_100q['lengths'],
                therm_cond=[coax_co_params[c]['tc']]*3,
                therm_scheme=therm_scheme_drive_10,
                stage_temps=fridge_100q['temps'])

    p_flux = passive_load(stage_labels=fridge_100q['labels'], 
                diameters=coax_co_params[c]['d'],
                lengths=fridge_100q['lengths'],
                therm_cond=[coax_co_params[c]['tc']]*3,
                therm_scheme=therm_scheme_flux_10,
                stage_temps=fridge_100q['temps'])

    a_ac = active_load_AC(stage_labels=fridge_100q['labels'], 
            signal_p=1e-3*10**(-5/10), 
            signal_f=6, 
            att=[10, 10, 0, 20, 20],
            cable_att=coax_co_params[c]['att_4'],
            lengths=fridge_100q['lengths'])

    a_dc = active_load_DC(i_in=20e-3, 
                    stage_labels=fridge_100q['labels'], 
                    att=[10, 10, 0, 0, 0], 
                    cable_rho=coax_co_params[c]['rho'], 
                    lengths=fridge_100q['lengths'], 
                    diameters=coax_co_params[c]['d'])

    heat_data_10.append([p_drive, p_flux, a_ac, a_dc])

### Comparison Plot

In [ ]:
n_drive = 14
n_flux = 14
dc_drive = 0.33
dc_flux = 1

edited_loads = [[di[0]*n_drive, 
                 di[1]*n_flux, 
                 di[2]*dc_drive*n_drive, 
                 di[3]*dc_flux*n_flux]
                for di in data]

edited_loads_10 = [[di[0]*n_drive, 
                 di[1]*n_flux, 
                 di[2]*dc_drive*n_drive, 
                 di[3]*dc_flux*n_flux]
                for di in heat_data_10]

figs = []
colours = ['red', 'orange', 'green', 'blue']

#iterating stages
for i in range(5):

    fig = plt.figure(figsize=(7, 5))
    figs.append(fig)
    ax = plt.axes()

    #iterating cables
    for ii in range(8):
        #iterating load type
        bot = 0
        bot_10 = 0
        for iii in range(4):
            val = edited_loads[ii][iii][i] / fridge_ours['cool_p'][i]
            plt.bar(x=1+3*ii, 
                    height=val, 
                    width=1, 
                    bottom=bot,
                    color=colours[iii])
            bot += val

        for iii in range(4):
            val_10 = edited_loads_10[ii][iii][i] / fridge_ours['cool_p'][i]
            plt.bar(x=2+3*ii, 
                    height=val_10, 
                    width=1, 
                    bottom=bot_10,
                    color=colours[iii])
            bot_10 += val_10

    plt.title('Heat Loads on ' + fridge_100q['labels'][i] + ' Stage\n' +
                'No. Drive = ' + str(n_drive) + ', No. Flux = ' + str(n_flux))
    plt.xlabel('Cable Types')
    plt.ylabel(r'$P_{total}/P_{cooling}$')
    leg_labels = ['Passive (Drive)', 'Passive (Flux)', 'Active (Drive)', 'Active (Flux)']
    leg = plt.legend(leg_labels)
    plt.legend(leg.legendHandles[::-1], leg_labels[::-1], loc=(1.01, 0.75))

    plt.xticks([1+3*i for i in range(8)] + [2+3*i for i in range(8)], 
                [c + r' $A_{50K}=0$' for c in cables] + [c + r' $A_{50K}=10$' for c in cables],
                rotation=90)

plt.show()

### Noise Plots

In [ ]:
noise_data_d = [drive_noise(f=6e9, 
                      stage_labels=fridge_ours['labels'], 
                      stage_temps=fridge_ours['temps'], 
                      att=[0, 20, 0, 20, 20], 
                      cable_att=[0, 0, 0, 0, 0], 
                      lengths=fridge_ours['lengths'])[-1],
                drive_noise(f=6e9, 
                      stage_labels=fridge_ours['labels'], 
                      stage_temps=fridge_ours['temps'], 
                      att=[10, 10, 0, 20, 20], 
                      cable_att=[0, 0, 0, 0, 0], 
                      lengths=fridge_ours['lengths'])[-1]]

noise_data_f = [np.sqrt(flux_noise(f=6e9, 
                      stage_labels=fridge_ours['labels'], 
                      stage_temps=fridge_ours['temps'], 
                      att=[0, 20, 0, 0, 0], 
                      cable_att=[0, 0, 0, 0, 0], 
                      lengths=fridge_ours['lengths'])[-1]),
                np.sqrt(flux_noise(f=6e9, 
                      stage_labels=fridge_ours['labels'], 
                      stage_temps=fridge_ours['temps'], 
                      att=[10, 10, 0, 0, 0], 
                      cable_att=[0, 0, 0, 0, 0], 
                      lengths=fridge_ours['lengths'])[-1])]

In [ ]:
fig = plt.figure(figsize=(7, 5))
ax1 = plt.axes()
ax2 = ax1.twinx()

ax1.bar(1, noise_data_d[0], width=1, color='red')
ax1.bar(2, noise_data_d[1], width=1, color='blue')
ax2.bar([4, 5], noise_data_f, width=1, color=['red', 'blue'])

ax2.set_ylim(0, max(noise_data_f)*1.05)

plt.xticks([1.5, 4.5], ['Drive Line\nNoise Photons', 'Flux Line\nNoise Current'])
ax1.legend([r'$A_{40K}=20dB, A_{50K}=0dB$', r'$A_{4K}=0dB, A_{50K}=10dB$'], loc='lower left')
plt.title("Drive and Flux Line Noise")
ax1.set_ylabel(r'Noise Photon Number ($Hz^{-1}s^{-1}$)')
ax2.set_ylabel(r'Noise Current ($A/\sqrt{Hz}$)')

plt.show()

## Full Fridge Setup

### Image

In [ ]:
ipd.Image('D:\\Uni\\Honours\\GitLab\\DiLonardo_Adrien\\' +
            '2021 Honours - Dilution Fridge\\Numerical Modeling\\' +
            'Optimisation Figures and Code\\Cable Setup Images\\Full.PNG')

### Data

In [2]:
therm_scheme_drive = [[True, True, False, True, True], #inner
                  [True, True, False, True, True], #dielectric
                  [True, True, True, True, True]]   #outer

therm_scheme_flux = [[True, True, False, False, True], #inner
                  [True, True, False, False, True], #dielectric
                  [True, True, True, True, True]]   #outer

therm_scheme_output = [[True, True, False, False, True], #inner
                  [True, True, False, False, True], #dielectric
                  [True, True, True, True, True]]   #outer

In [3]:
c_drive = '119-SS-SS'
c_flux = '119-SS-SS'
c_out_upper = '119-AgCuNi-CuNi'
c_out_lower = '119-NbTi-NbTi'

p_drive = passive_load(stage_labels=fridge_ours['labels'], 
        diameters=coax_co_params[c_drive]['d'],
        lengths=fridge_ours['lengths'],
        therm_cond=[coax_co_params[c_drive]['tc'], lambda t: 0, coax_co_params[c_drive]['tc']],
        therm_scheme=therm_scheme_drive,
        stage_temps=fridge_ours['temps'])

p_flux = passive_load(stage_labels=fridge_ours['labels'], 
        diameters=coax_co_params[c_flux]['d'],
        lengths=fridge_ours['lengths'],
        therm_cond=[coax_co_params[c_flux]['tc'], lambda t: 0, coax_co_params[c_flux]['tc']],
        therm_scheme=therm_scheme_flux,
        stage_temps=fridge_ours['temps'])

p_out_lower = passive_load(stage_labels=fridge_100q['labels'], 
        diameters=coax_co_params[c_out_lower]['d'],
        lengths=fridge_100q['lengths'],
        therm_cond=[coax_co_params[c_out_lower]['tc'], lambda t: 0, coax_co_params[c_out_lower]['tc']],
        therm_scheme=therm_scheme_output,
        stage_temps=fridge_ours['temps'])

p_out_upper = passive_load(stage_labels=fridge_ours['labels'], 
        diameters=coax_co_params[c_out_upper]['d'],
        lengths=fridge_ours['lengths'],
        therm_cond=[coax_co_params[c_out_upper]['tc'], lambda t: 0, coax_co_params[c_out_upper]['tc']],
        therm_scheme=therm_scheme_output,
        stage_temps=fridge_ours['temps'])

p_out = p_out_upper[0:2].append(p_out_lower[2:])

a_ac = active_load_AC(stage_labels=fridge_ours['labels'], 
        signal_p=1e-3*10**(-5/10), 
        signal_f=6, 
        att=[10, 10, 0, 20, 20],
        cable_att=coax_co_params[c_drive]['att_4'],
        lengths=fridge_ours['lengths'])

a_dc = active_load_DC(i_in=20e-3, 
                stage_labels=fridge_ours['labels'], 
                att=[10, 10, 0, 0, 0], 
                cable_rho=coax_co_params[c_flux]['rho'], 
                lengths=fridge_ours['lengths'], 
                diameters=coax_co_params[c_flux]['d'])


In [4]:
c_p = np.array(fridge_ours['cool_p'])
load_matrix_SS = pd.DataFrame(
    [p_drive*14/c_p, p_flux*14/c_p, p_out*4/c_p, a_ac*14*0.33/c_p, a_dc*14*1/c_p], 
    index=['Passive (Drive)', 'Passive (Flux)', 'Passive (Output)', 'Active (Drive)', 'Active (Flux)']
)
load_matrix_SS

,50K,4K,Still,CP,MXC
Passive (Drive),0.008571,0.002867,0.000311,0.010104,0.001145
Passive (Flux),0.008571,0.002867,0.000311,0.006567,0.036557
Passive (Output),0.019293,0.006455,0.000061,0.000946,0.007222
Active (Drive),0.000138,0.000152,0.000060,0.005030,0.000512
Active (Flux),0.025200,0.050400,0.000000,0.219299,2.111311


In [5]:
pd.DataFrame(
    [p_drive, p_flux, p_out, a_ac*0.33, a_dc], 
    index=['Passive (Drive)', 'Passive (Flux)', 'Passive (Output)', 'Active (Drive)', 'Active (Flux)']
)

,50K,4K,Still,CP,MXC
Passive (Drive),0.006122,0.000102,6.659572e-07,2.165147e-07,1.636287e-09
Passive (Flux),0.006122,0.000102,6.659572e-07,1.407271e-07,5.222393e-08
Passive (Output),0.048233,0.000807,4.564834e-07,7.092795e-08,3.610935e-08
Active (Drive),0.000099,0.000005,1.276981e-07,1.077860e-07,7.314197e-10
Active (Flux),0.018000,0.001800,0.000000e+00,4.699262e-06,3.016159e-06


In [ ]:
c_drive = '119-CuNi-CuNi'
c_flux = '119-CuNi-CuNi'
c_out_upper = '119-AgCuNi-CuNi'
c_out_lower = '119-NbTi-NbTi'

p_drive = passive_load(stage_labels=fridge_ours['labels'], 
            diameters=coax_co_params[c_drive]['d'],
            lengths=fridge_ours['lengths'],
            therm_cond=[coax_co_params[c_drive]['tc'], lambda t: 0, coax_co_params[c_drive]['tc']],
            therm_scheme=therm_scheme_drive,
            stage_temps=fridge_ours['temps'])

p_flux = passive_load(stage_labels=fridge_ours['labels'], 
            diameters=coax_co_params[c_flux]['d'],
            lengths=fridge_ours['lengths'],
            therm_cond=[coax_co_params[c_flux]['tc'], lambda t: 0, coax_co_params[c_flux]['tc']],
            therm_scheme=therm_scheme_flux,
            stage_temps=fridge_ours['temps'])

p_out_lower = passive_load(stage_labels=fridge_100q['labels'], 
            diameters=coax_co_params[c_out_lower]['d'],
            lengths=fridge_100q['lengths'],
            therm_cond=[coax_co_params[c_out_lower]['tc'], lambda t: 0, coax_co_params[c_out_lower]['tc']],
            therm_scheme=therm_scheme_output,
            stage_temps=fridge_ours['temps'])

p_out_upper = passive_load(stage_labels=fridge_ours['labels'], 
            diameters=coax_co_params[c_out_upper]['d'],
            lengths=fridge_ours['lengths'],
            therm_cond=[coax_co_params[c_out_upper]['tc'], lambda t: 0, coax_co_params[c_out_upper]['tc']],
            therm_scheme=therm_scheme_output,
            stage_temps=fridge_ours['temps'])

p_out = p_out_upper[0:2].append(p_out_lower[2:])

a_ac = active_load_AC(stage_labels=fridge_ours['labels'], 
        signal_p=1e-3*10**(-5/10), 
        signal_f=6, 
        att=[10, 10, 0, 20, 20],
        cable_att=coax_co_params[c_drive]['att_4'],
        lengths=fridge_ours['lengths'])

a_dc = active_load_DC(i_in=20e-3, 
                stage_labels=fridge_ours['labels'], 
                att=[10, 10, 0, 0, 0], 
                cable_rho=coax_co_params[c_flux]['rho'], 
                lengths=fridge_ours['lengths'], 
                diameters=coax_co_params[c_flux]['d'])

In [ ]:
c_p = np.array(fridge_ours['cool_p'])
load_matrix_CuNi = pd.DataFrame(
    [p_drive*14/c_p, p_flux*14/c_p, p_out*4/c_p, a_ac*14*0.33/c_p, a_dc*14*1/c_p], 
    index=['Passive (Drive)', 'Passive (Flux)', 'Passive (Output)', 'Active (Drive)', 'Active (Flux)']
)
load_matrix_CuNi

### Plot

In [ ]:
#one cable type on its own - change 'load_matrix' to the relevant cable load matrix
colour = ['red', 'orange', 'yellowgreen', 'green', 'blue']

##iterate stage
for i in range(5):
    bot = 0
    #iterate load type
    for ii in range(5):
        val = load_matrix.iloc[ii, i]
        plt.bar(2*i+1, load_matrix.iloc[ii, i], width=1, bottom=bot, color=colour[ii])
        bot += val

plt.title('Heat Loads on Each Stage')
plt.xlabel('Stage')
plt.ylabel(r'$P/P_{cooling}$')

plt.xticks([1, 3, 5, 7, 9], fridge_ours['labels'])
plt.legend(['Passive (Drive)', 'Passive (Flux)', 'Passive (Output)', 'Active (Drive)', 'Active (Flux)'])

plt.savefig('full_setup_SS_drive.png', dpi=200)
plt.show()

In [ ]:
#comparison plot - run the data for both cable sets
colour = ['red', 'orange', 'yellowgreen', 'green', 'blue']

##iterate stage
for i in range(5):
    bot = 0
    #iterate load type
    for ii in range(5):
        val = load_matrix_SS.iloc[ii, i]
        plt.bar(3*i+1, val, width=1, bottom=bot, color=colour[ii])
        bot += val

    bot = 0
    for ii in range(5):
        val = load_matrix_CuNi.iloc[ii, i]
        plt.bar(3*i+2, val, width=1, bottom=bot, color=colour[ii])
        bot += val

plt.title('Heat Loads on Each Stage')
plt.xlabel('Stage')
plt.ylabel(r'$P/P_{cooling}$')

plt.xticks([1, 4, 7, 10, 13] + [2, 5, 8, 11, 14], 
           [l + '- SS' for l in fridge_ours['labels']] + [l + '- CuNi' for l in fridge_ours['labels']],
           rotation=90)
plt.legend(['Passive (Drive)', 'Passive (Flux)', 'Passive (Output)', 'Active (Drive)', 'Active (Flux)'])

# plt.savefig('full_setup_SS_drive.png', dpi=200)
plt.show()

### CuNi for lower drive and lower flux

#### CP-MXC

In [ ]:
total_load_matrix_CP_MXC = pd.concat([load_matrix_SS[['50K', '4K', 'Still', 'CP']], load_matrix_CuNi[['MXC']]], 
                    axis=1)
total_load_matrix_CP_MXC.iloc[1, 4] = 0.0401
total_load_matrix_CP_MXC

#### Still-MXC

In [ ]:
#process load matrix SS and load matrix CuNi first
total_load_matrix_Still_MXC = pd.concat([load_matrix_SS[['50K', '4K', 'Still']], load_matrix_CuNi[['CP', 'MXC']]], 
                    axis=1)
total_load_matrix_Still_MXC.iloc[0, 3] = 0.01255
total_load_matrix_Still_MXC.iloc[1, 4] = 0.0425
total_load_matrix_Still_MXC

In [ ]:
colour = ['red', 'orange', 'yellowgreen', 'green', 'blue']

##iterate stage
for i in range(5):
    bot = 0
    #iterate load type
    for ii in range(5):
        val = total_load_matrix.iloc[ii, i]
        plt.bar(2*i+1, total_load_matrix.iloc[ii, i], width=1, bottom=bot, color=colour[ii])
        bot += val

plt.title('Heat Loads on Each Stage')
plt.xlabel('Stage')
plt.ylabel(r'$P/P_{cooling}$')

plt.xticks([1, 3, 5, 7, 9], fridge_ours['labels'])
plt.legend(['Passive (Drive)', 'Passive (Flux)', 'Passive (Output)', 'Active (Drive)', 'Active (Flux)'])

plt.savefig('full_setup_SS_drive.png', dpi=200)
plt.show()

In [ ]:
#comparison plot - run the data for both cable sets
colour = ['red', 'orange', 'yellowgreen', 'green', 'blue']

##iterate stage
for i in range(5):
    bot = 0
    #iterate load type
    for ii in range(5):
        val = total_load_matrix_CP_MXC.iloc[ii, i]
        plt.bar(3*i+1, val, width=1, bottom=bot, color=colour[ii])
        bot += val

    bot = 0
    for ii in range(5):
        val = total_load_matrix_Still_MXC.iloc[ii, i]
        plt.bar(3*i+2, val, width=1, bottom=bot, color=colour[ii])
        bot += val

plt.title('Heat Loads on Each Stage')
plt.xlabel('Stage')
plt.ylabel(r'$P/P_{cooling}$')

plt.xticks([1, 4, 7, 10, 13] + [2, 5, 8, 11, 14], 
           [l + '- CP-MXC' for l in fridge_ours['labels']] + [l + '- Still-MXC' for l in fridge_ours['labels']],
           rotation=90)
plt.legend(['Passive (Drive)', 'Passive (Flux)', 'Passive (Output)', 'Active (Drive)', 'Active (Flux)'])

# plt.savefig('full_setup_SS_drive.png', dpi=200)
plt.show()

In [ ]:
#comparison 2
colour = ['red', 'orange', 'yellowgreen', 'green', 'blue']

##iterate stage
for i in range(5):
    bot = 0
    #iterate load type
    for ii in range(5):
        val = total_load_matrix_CP_MXC.iloc[ii, i]
        plt.bar(4*i+1, val, width=1, bottom=bot, color=colour[ii])
        bot += val

    bot = 0
    for ii in range(5):
        val = total_load_matrix_Still_MXC.iloc[ii, i]
        plt.bar(4*i+2, val, width=1, bottom=bot, color=colour[ii])
        bot += val

    bot = 0
    for ii in range(5):
        val = load_matrix_CuNi.iloc[ii, i]
        plt.bar(4*i+3, val, width=1, bottom=bot, color=colour[ii])
        bot += val


plt.title('Heat Loads on Each Stage')
plt.xlabel('Stage')
plt.ylabel(r'$P/P_{cooling}$')

plt.xticks([4*i+1 for i in range(5)] + [4*i+2 for i in range(5)] + [4*i+3 for i in range(5)], 
           [l + '- CP-MXC' for l in fridge_ours['labels']] 
           + [l + '- Still-MXC' for l in fridge_ours['labels']]
           + [l + '- Full CuNi' for l in fridge_ours['labels']],
           rotation=90)
plt.legend(['Passive (Drive)', 'Passive (Flux)', 'Passive (Output)', 'Active (Drive)', 'Active (Flux)'])

# plt.savefig('full_setup_SS_drive.png', dpi=200)
plt.show()

## Passive + Active Flux, All Stages, All Cables, One Plot

In [ ]:
# Parameters

therm_scheme 

## Testing

In [ ]:
m_SS = coax_co_params['119-SS-SS']['tc'](1)
m_CN = coax_co_params['119-CuNi-CuNi']['tc'](1)
L_SS = fridge_ours['lengths'][2]
L_CN = fridge_ours['lengths'][3]

np.sqrt((4**2*m_SS/L_SS + 0.1**2*m_CN/L_CN)/(m_CN/L_CN + m_SS/L_SS))

In [ ]:
np.sqrt((4**2*m_SS/L_SS + 0.1**2*m_SS/L_CN)/(m_SS/L_CN + m_SS/L_SS))

In [ ]:
np.sqrt((4**2*m_CN/L_SS + 0.1**2*m_SS/L_CN)/(m_SS/L_CN + m_CN/L_SS))

In [ ]:
mxc_flux_out = (
    cs_areas['119'][2] / fridge_ours['lengths'][-1] *
    coax_co_params['119-CuNi-CuNi']['tc'](1) / 2 *
    (fridge_ours['temps'][3]**2 - fridge_ours['temps'][4]**2)
)

alpha_1 = (
    (cs_areas['119'][0] + cs_areas['119'][1]) * coax_co_params['119-CuNi-CuNi']['tc'](1) / 
    (fridge_ours['lengths'][2] + fridge_ours['lengths'][3])
)

alpha_2 = (
    (cs_areas['119'][0] + cs_areas['119'][1]) * coax_co_params['119-CuNi-CuNi']['tc'](1) / 
    fridge_ours['lengths'][4]
)

mxc_flux_in = (
    alpha_1 * alpha_2 /
    (2 * (alpha_1 + alpha_2)) *
    (fridge_ours['temps'][1]**2 - fridge_ours['temps'][4]**2)
)

(mxc_flux_out + mxc_flux_in) * 14 / 20e-6

In [ ]:
p_flux*14

In [ ]:
outer =  (
    cs_areas['119'][2] / fridge_ours['lengths'][1] *
    coax_co_params['119-CuNi-CuNi']['tc'](1) / 2 *
    (fridge_ours['temps'][0]**2 - fridge_ours['temps'][1]**2)
)
inner = (
    (cs_areas['119'][0]+cs_areas['119'][1]) / fridge_ours['lengths'][1] *
    coax_co_params['119-CuNi-CuNi']['tc'](1) / 2 *
    (fridge_ours['temps'][0]**2 - fridge_ours['temps'][1]**2)
)

outer+inner

In [ ]:
(outer+inner-p_flux[1])/p_flux[1]

In [ ]:
A = (cs_areas['119'][2] + cs_areas['119'][1] + cs_areas['119'][0])
L = fridge_ours['lengths'][1]
m = coax_co_params['119-CuNi-CuNi']['tc'](4)

(A-np.pi*(1.19/2/1000)**2)/A, L, m

In [ ]:
therm_cond_coaxco['119-CuNi-CuNi'](4)

In [ ]:
1.74e-5/100 / (np.pi*(1.19/2/1000)**2)

In [ ]:
(np.pi*(1.19/2/1000)**2)

In [ ]:
1.74e-5/100 / sum(cs_areas['119'])

In [ ]:
p_flux

In [ ]:
coax_co_params['119-SS-SS']['tc'](1) * (cs_areas['119'][0] + cs_areas['119'][2])

In [ ]:
coax_co_params['119-CuNi-CuNi']['tc'](4) * cs_areas['119'][2]

In [ ]:
coax_co_params['119-CuNi-CuNi']['tc'](1)

In [ ]:
coax_co_params['119-SS-SS']['tc'](1)

In [ ]:
cs_areas['119'][0], cs_areas['119'][2]